<div class="notebook-buttons" style="display:flex; padding-top: 5rem;padding-bottom: 2.5rem;line-height: 2.15;">
    <a href="https://colab.research.google.com/github/zazuko/notebooks/blob/master/notebooks/politics/parliament_commissions.ipynb">
        <div id="colab-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; align-items: center;">
            <img class="call-to-action-img" src="img/colab.svg" width="30" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Run in Google Colab</div>
        </div>
    </a>
    <a href="https://raw.githubusercontent.com/zazuko/notebooks/master/notebooks/politics/parliament_commissions.ipynb" download>
        <div id="download-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img/download.svg" width="22" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Download Notebook</div>
        </div>
    </a>
    <a href="https://github.com/zazuko/notebooks/blob/master/notebooks/politics/parliament_commissions.ipynb">
        <div id="github-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img/github.svg" width="25" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">View on GitHub</div>
        </div>
    </a>
</div>

# Swiss Parliament: Commissions

The Federal Chancellery maintains data on the Swiss political system. The curia dataset is publicly available it provides data on political parties, parliamentary comissions, members of parliament and their affiliations. 

Parliament data is also available as [Linked Data](https://en.wikipedia.org/wiki/Linked_data). 

## Setup

### SPARQL endpoint

Swiss political data can be accessed with [SPARQL queries](https://www.w3.org/TR/rdf-sparql-query/).   
You can send queries using HTTP requests. The API endpoint is **[https://lindas.admin.ch/query/](https://lindas.admin.ch/query).**  

### SPARQL client

We will use the `SparqlClient` from [graphly](https://github.com/zazuko/graphly) to communicate with the database. 
Graphly will allow us to:
* send SPARQL queries
* automatically add prefixes to all queries
* format response to `pandas` or `geopandas`

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import warnings

from plotly.subplots import make_subplots
from graphly.api_client import SparqlClient

%matplotlib inline
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

In [ ]:
# Uncomment to install dependencies in Colab environment
#!pip install git+https://github.com/zazuko/graphly.git

In [ ]:
sparql = SparqlClient("https://int.lindas.admin.ch/query")

sparql.add_prefixes({
    "schema": "<http://schema.org/>"
})

SPARQL queries can become very long. To improve the readibility, we will work wih [prefixes](https://en.wikibooks.org/wiki/SPARQL/Prefixes).
 
Using the `add_prefixes` method, we can define persistent prefixes. 
Every time you send a query, `graphly` will now automatically add the prefixes for you.

# Political Parties

In [ ]:
query = """
SELECT ?council ?faction (COUNT(?mep) AS ?members)
FROM <https://lindas.admin.ch/fch/curia>
FROM <https://lindas.admin.ch/fch/rvov>
WHERE {

  	VALUES ?_council { <https://politics.ld.admin.ch/council/S> <https://politics.ld.admin.ch/council/N> <https://politics.ld.admin.ch/council/UFA>}
	?mep a schema:Person;
         schema:memberOf/schema:memberOf ?_council;
                        schema:memberOf ?role.
  
   FILTER NOT EXISTS { ?role schema:endDate ?end }
  
   ?role schema:memberOf ?_faction.
   ?_faction a <http://schema.org/ParliamentaryGroup>;
             schema:name ?faction.
  
   ?_council schema:name ?council.
   FILTER (lang(?council) = 'de')
   FILTER (lang(?faction) = 'de')
}
GROUP BY ?council ?faction
ORDER BY DESC(?council) DESC (?members)
"""

df = sparql.send_query(query)

In [ ]:
N_FACTIONS=df.faction.nunique()
cmap = plt.get_cmap('YlOrRd')
colors = [mcolors.rgb2hex(cmap((i+1)/N_FACTIONS)) for i in range(N_FACTIONS)]
factions = [
     "Grünliberale Fraktion",
     "Grüne Fraktion",
     "Die Mitte-Fraktion. Die Mitte. EVP.",
     "Sozialdemokratische Fraktion",
     "FDP-Liberale Fraktion",
     "Fraktion der Schweizerischen Volkspartei"
]
colormap = dict(zip(factions,colors))

fig = make_subplots(rows=1, cols=3, subplot_titles=df["council"].unique(), specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}]])
for i, council in enumerate(df["council"].unique()):

     members = dict(zip(df[df.council == council]["faction"],df[df.council == council]["members"]))

     fig.add_trace(go.Pie(
          values=[members[f] if f in members else 0 for f in factions],
          labels=factions, sort=False
          ), row=1, col=i+1)
     fig.update_traces(textinfo='none', marker={"colors": [colormap[f] for f in factions]})

fig.update_annotations(yshift=-280)
fig.update_layout(height=400, title={"text": "Members of Parliament (as of today)", "x": 0.5})
fig.show()

# Parliamentary Committees

In [ ]:
query = """
SELECT ?council ?committee (COUNT(?mep) AS ?members)
FROM <https://lindas.admin.ch/fch/curia>
FROM <https://lindas.admin.ch/fch/rvov>
WHERE {

  	VALUES ?_council { <https://politics.ld.admin.ch/council/S> <https://politics.ld.admin.ch/council/N>}
	?mep a schema:Person;
         schema:memberOf/schema:memberOf ?_council;
                        schema:memberOf ?role.
  
   FILTER NOT EXISTS { ?role schema:endDate ?end }
  
   ?role schema:memberOf ?_committee.
   ?_committee a <http://schema.org/ParliamentaryCommittee>;
             schema:additionalType schema:MainCommittee;
             schema:name ?committee.
  
   ?_council schema:name ?council.
   FILTER (lang(?council) = 'de')
   FILTER (lang(?committee) = 'de')
}
GROUP BY ?council ?committee
ORDER BY DESC (?members)
"""

df = sparql.send_query(query)

In [ ]:
px.treemap(df, path=["council", "committee"], values="members")

The above chart shows the number of members per committee.

# Commissions and Factions

In [ ]:
query = """
SELECT ?faction (COUNT(?mep) AS ?members)
FROM <https://lindas.admin.ch/fch/curia>
WHERE {

	?mep a schema:Person;
         schema:memberOf/schema:memberOf <https://politics.ld.admin.ch/council/UFA>;
                        schema:memberOf ?role.
  
   FILTER NOT EXISTS { ?role schema:endDate ?end }
  
   ?role schema:memberOf ?_faction.
   ?_faction a <http://schema.org/ParliamentaryGroup>;
             schema:name ?faction.
  
   FILTER (lang(?faction) = 'de')
}
GROUP BY ?faction
ORDER BY DESC(?faction) DESC (?members)
"""

df_members = sparql.send_query(query)

In [ ]:
query = """
SELECT ?faction (count(?_committee) as ?committees)
FROM <https://lindas.admin.ch/fch/curia>
WHERE {

	?mep a schema:Person;
         schema:memberOf ?ufa_role;
                        schema:memberOf ?committee_role;
                        schema:memberOf ?faction_role.
  
   ?ufa_role schema:memberOf <https://politics.ld.admin.ch/council/UFA>.
  
   FILTER NOT EXISTS { ?ufa_role schema:endDate ?end }
  
   ?faction_role schema:memberOf ?_faction.
   ?committee_role schema:memberOf ?_committee.
   ?_faction a <http://schema.org/ParliamentaryGroup>;
             schema:name ?faction.
   ?_committee a schema:ParliamentaryCommittee;
             schema:additionalType schema:MainCommittee.
  
   FILTER (lang(?faction) = 'de')
}
GROUP BY ?faction
ORDER BY DESC (?count)
"""

df_committees = sparql.send_query(query)

In [ ]:
df = pd.merge(df_committees,df_members, on='faction', how='left')
df["metric"] = df["committees"]/df["members"]
df.sort_values(by="metric", ascending=False, inplace=True)
df

In [ ]:
fig = px.bar(df, x="faction", y="metric", hover_data=["committees", "members"], labels={'metric':'Affiliations per member', "faction": "Faction", "committees": "Committee affiliations","members": "Members"})

fig.update_layout(
    title='Committee Affiliations', 
    title_x=0.5,
    yaxis_title="Affiliations per member",
    xaxis_title=""
)
fig.show()

The above chart shows in how many committees members of the different parties are engaged in on average.